In [2]:
import sys
sys.path.append('..') # for import src

import os
import cloudpickle
import lzma
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_predict
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import lightgbm as lgb
import talib

import src
from src.ml_utils import (
    fetch_ohlcv, 
    visualize_result, 
    normalize_position, 
    calc_position_cv,
    get_feature_columns,
    get_symbols,
    unbiased_rank,
    ewm_finite,
)
cloudpickle.register_pickle_by_value(src) # for model portability

In [39]:
# symbols = 'BTC,ETH'.split(',')
symbols = os.getenv('ALPHAPOOL_SYMBOLS').split(',') # 売買代金が多く、古いもの
df = fetch_ohlcv(symbols=symbols, with_target=True)
df.to_pickle('/tmp/df.pkl')
display(df)

op            hi            lo  \
timestamp                 symbol                                             
2019-07-20 00:00:00+00:00 BNB        29.074200     29.251700     28.981150   
                          BTC     10532.400561  10596.176732  10451.465293   
                          ETH       221.191514    224.079639    221.015166   
                          XRP         0.319763      0.323658      0.319140   
2019-07-20 01:00:00+00:00 BNB        29.102050     30.051750     28.937300   
...                                        ...           ...           ...   
2022-09-02 08:00:00+00:00 ETH      1594.401429   1596.874286   1585.030000   
                          LINK        6.882598      6.912045      6.853060   
                          MATIC       0.881401      0.890773      0.880405   
                          SOL        31.624165     31.658390     31.415591   
                          XRP         0.331690      0.331690      0.330483   

                                            cl        execution_start_at  \
timestamp                 symbol                                           
2019-07-20 00:00:00+00:00 BNB        29.102300 2019-07-20 01:00:00+00:00   
                          BTC     10466.256172 2019-07-20 01:00:00+00:00   
                          ETH       221.450416 2019-07-20 01:00:00+00:00   
                          XRP         0.319678 2019-07-20 01:00:00+00:00   
2019-07-20 01:00:00+00:00 BNB        30.036500 2019-07-20 02:00:00+00:00   
...                                        ...                       ...   
2022-09-02 08:00:00+00:00 ETH      1590.434286 2022-09-02 09:00:00+00:00   
                          LINK        6.873536 2022-09-02 09:00:00+00:00   
                          MATIC       0.890107 2022-09-02 09:00:00+00:00   
                          SOL        31.562069 2022-09-02 09:00:00+00:00   
                          XRP         0.330670 2022-09-02 09:00:00+00:00   

                                       ret  
timestamp                 symbol            
2019-07-20 00:00:00+00:00 BNB     0.063542  
                          BTC     0.034167  
                          ETH     0.034369  
                          XRP     0.043545  
2019-07-20 01:00:00+00:00 BNB     0.016383  
...                                    ...  
2022-09-02 08:00:00+00:00 ETH          NaN  
                          LINK         NaN  
                          MATIC        NaN  
                          SOL          NaN  
                          XRP          NaN  

[245753 rows x 6 columns]

In [5]:
class DebugModel:
    def __init__(self):
        pass

    def fit(self, df):
        print('fit df.shape', df.shape)
        
    def predict(self, df):
        print('predict df.shape', df.shape)
        
        df = df.copy()
        df['position'] = 0.0
        
        return df['position']

In [6]:
from src.ml_utils import _calc_cv_indicies
df = pd.read_pickle('/tmp/df_price_type_none.pkl')

print(_calc_cv_indicies(df))

model = DebugModel()
calc_position_cv(model, df)

[(MultiIndex([('2020-03-07 00:00:00+00:00',   'ADA'),
            ('2020-03-07 00:00:00+00:00',  'ATOM'),
            ('2020-03-07 00:00:00+00:00',   'BNB'),
            ('2020-03-07 00:00:00+00:00',   'BTC'),
            ('2020-03-07 00:00:00+00:00',   'ETH'),
            ('2020-03-07 00:00:00+00:00',  'LINK'),
            ('2020-03-07 00:00:00+00:00', 'MATIC'),
            ('2020-03-07 00:00:00+00:00',   'XRP'),
            ('2020-03-07 01:00:00+00:00',   'ADA'),
            ('2020-03-07 01:00:00+00:00',  'ATOM'),
            ...
            ('2022-09-09 16:00:00+00:00',   'ADA'),
            ('2022-09-09 16:00:00+00:00',  'ATOM'),
            ('2022-09-09 16:00:00+00:00',   'BNB'),
            ('2022-09-09 16:00:00+00:00',   'BTC'),
            ('2022-09-09 16:00:00+00:00',   'DOT'),
            ('2022-09-09 16:00:00+00:00',   'ETH'),
            ('2022-09-09 16:00:00+00:00',  'LINK'),
            ('2022-09-09 16:00:00+00:00', 'MATIC'),
            ('2022-09-09 16:00:00+00:00',   'S